## Budget Classifier

#### Setup and Context

In this notebook, I will be using transaction history data pulled from a Nigerian bank's API to train a budget classifier. The aim is to try and understand trends in spending and if possible, eventually be able to provide some sort of trained model per account that predicts spending trends.

This seems pretty trivial in the western world where majority of spending is done via credit and debit cards. Most transaction history data has enough information to adequately classify a transaction into buckets such as transportation, entertainment, groceries, etc. A couple of banks in canada have even rolled out this feature as some sort of monthly account summary. The biggest challenge i presume they would have is classifying things bought at mega stores such as Walmart or Costco. They would generally be classified as groceries when infact, one could have been shopping for clothes or other household items.

Another challenge would be the miscellaneous cash withdrawals that cannot be classified because no one really knows what withdrawn cash is used for. Majority of the average Nigerian's spending is based on cash withdrawals so one would doubt the potency of classification techniques to build a sort of monthly summary for an account. Is there a way to extract more information from transaction histories? Could features like ATM location crossed with time of transaction be useful to predict trends in withdrawals? These are the questions im hoping to answer with this project.

first import necessary dependencies

In [1]:
import requests
import json
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

Import json data from rest API

In [2]:
payload = "{\"Envelope\":{\"xmlns\":\"http://schemas.xmlsoap.org/soap/envelope/\",\"Body\":{\"GetData\":{\"xmlns\":\"http://tempuri.org/\",\"AccountNumber\":\"0122863683\",\"StartDate\":\"2015-05-30T09:00:00\",\"EndDate\":\"2017-05-30T09:00:00\"}}}}"

headers = {
    'x-ibm-client-id': "6846f45b-129c-44cc-9a3a-6d37c35a7a89",
    'content-type': "application/json",
    'accept': "application/json"
    }

r = requests.post('https://api.gtbank.com/GetTransactionHistory', data=payload, headers=headers)
trans_dict = json.loads(r.text)

arrayy = trans_dict["Envelope"]["Body"]["GetDataResponse"]["GetDataResult"]["ResponseData"]["ALLTRANS"]["TransactionDetails"]["TransactionDetail"]

Fancy visualization of the Json data

In [3]:
data = pd.DataFrame(arrayy)
for i in arrayy:
    for a, b in i.items():
        print("{}: \t{}".format(a, b))
    print("\n")

Acctno: 	0122863683
Channel: 	Spend to Save  Contribution
Cr_Amt: 	0
Dr_Amt: 	36
Remark: 	Savings from expenses from 205/152573/1/1
Tra_Bal: 	225006.35
Tra_Date: 	5/31/2017 4:03:01 AM
Trans_Type: 	Spend to Save  Contribution
Val_Date: 	5/30/2017 4:03:01 AM


Acctno: 	0122863683
Channel: 	MEDICAL BILL/REFUND
Cr_Amt: 	50466.97
Dr_Amt: 	0
Remark: 	Medrefund May 2017
Tra_Bal: 	225042.35
Tra_Date: 	5/30/2017 3:37:21 PM
Trans_Type: 	MEDICAL BILL/REFUND
Val_Date: 	5/31/2017 3:37:21 PM


Acctno: 	0122863683
Channel: 	MISC.
Cr_Amt: 	106161.85
Dr_Amt: 	0
Remark: 	May 2017 Allow
Tra_Bal: 	174575.38
Tra_Date: 	5/30/2017 3:36:03 PM
Trans_Type: 	MISC.
Val_Date: 	5/31/2017 3:36:03 PM


Acctno: 	0122863683
Channel: 	MTHLY SALARY
Cr_Amt: 	68353.56
Dr_Amt: 	0
Remark: 	May 2017 Sal
Tra_Bal: 	68413.53
Tra_Date: 	5/30/2017 3:34:13 PM
Trans_Type: 	MTHLY SALARY
Val_Date: 	5/31/2017 3:34:13 PM


Acctno: 	0122863683
Channel: 	USSD
Cr_Amt: 	0
Dr_Amt: 	200
Remark: 	USSD-101CT0000000000361857077-2348081467539
Tra

Trans_Type: 	CASH WITHDRAWAL FROM OUR ATM
Val_Date: 	3/27/2017 8:39:32 PM


Acctno: 	0122863683
Channel: 	POS
Cr_Amt: 	0
Dr_Amt: 	3000
Remark: 	-118880- -TOTAL NIGERIA PLC      LA           LANG
Tra_Bal: 	57657.68
Tra_Date: 	3/27/2017 4:07:42 PM
Trans_Type: 	POS/WEB PURCHASE TRANSACTION
Val_Date: 	3/27/2017 4:07:42 PM


Acctno: 	0122863683
Channel: 	POS
Cr_Amt: 	0
Dr_Amt: 	2260
Remark: 	-003590- -GRAND SQ. SUPERMARKETANDSLA           LANG
Tra_Bal: 	60657.68
Tra_Date: 	3/27/2017 8:15:50 PM
Trans_Type: 	POS/WEB PURCHASE TRANSACTION
Val_Date: 	3/27/2017 8:15:50 PM


Acctno: 	0122863683
Channel: 	USSD
Cr_Amt: 	0
Dr_Amt: 	5000
Remark: 	USSD GTBank Transfer from : 08081467539FASHOLA OMOBOLAJI OLUWASEYI to BABATUNDE ABAYOMI ROTIMI                                                                            REF: 63626029091190286719636
Tra_Bal: 	62917.68
Tra_Date: 	3/27/2017 9:58:12 AM
Trans_Type: 	TRANSFER BETWEEN CUSTOMERS
Val_Date: 	3/27/2017 9:58:12 AM


Acctno: 	0122863683
Channel: 	USSD
Cr

Trans_Type: 	Airtime Purchase
Val_Date: 	1/23/2017 8:19:32 PM


Acctno: 	0122863683
Channel: 	Mobile Banking
Cr_Amt: 	500
Dr_Amt: 	0
Remark: 	MBANKING - OWN Account Transfer
Tra_Bal: 	607.37
Tra_Date: 	1/23/2017 8:18:03 PM
Trans_Type: 	OWN ACCOUNT TRANSFER
Val_Date: 	1/23/2017 8:18:03 PM


Acctno: 	0122863683
Channel: 	Spend to Save  Contribution
Cr_Amt: 	0
Dr_Amt: 	312
Remark: 	Savings from expenses from 205/152573/1/1
Tra_Bal: 	107.37
Tra_Date: 	1/20/2017 12:52:04 AM
Trans_Type: 	Spend to Save  Contribution
Val_Date: 	1/20/2017 12:52:04 AM


Acctno: 	0122863683
Channel: 	CASH WITHDRAWAL FROM OUR ATM
Cr_Amt: 	0
Dr_Amt: 	10000
Remark: 	-008869- -GTBank A/ Ademola       V/I         LANG
Tra_Bal: 	419.37
Tra_Date: 	1/20/2017 3:02:43 PM
Trans_Type: 	CASH WITHDRAWAL FROM OUR ATM
Val_Date: 	1/20/2017 3:02:43 PM


Acctno: 	0122863683
Channel: 	FlutterWave App
Cr_Amt: 	10000
Dr_Amt: 	0
Remark: 	000015170120140828000106882416  Bolaji FASHOLA OMOBOLAJI OLUWASEYI ADEREMI   OKEOWO  to  FASHOLA OM

Tra_Date: 	11/16/2016 8:22:33 AM
Trans_Type: 	CASH DEPOSIT
Val_Date: 	11/16/2016 8:22:33 AM


Acctno: 	0122863683
Channel: 	Spend to Save  Contribution
Cr_Amt: 	0
Dr_Amt: 	6000
Remark: 	Savings from expenses from 205/152573/1/1
Tra_Bal: 	103017.15
Tra_Date: 	11/15/2016 12:59:38 AM
Trans_Type: 	Spend to Save  Contribution
Val_Date: 	11/15/2016 12:59:38 AM


Acctno: 	0122863683
Channel: 	NIBSS Instant Payment Outward
Cr_Amt: 	0
Dr_Amt: 	200000
Remark: 	000013161115094648000027911098 Payment for hall for Bolaji Fashola and Cassandra TO FB/HYPER-CADE VENTURES LTD  from  FASHOLA OMOBOLAJI OLUWASEYI
Tra_Bal: 	109017.15
Tra_Date: 	11/15/2016 9:46:49 AM
Trans_Type: 	NIBSS Instant Payment Outward
Val_Date: 	11/15/2016 9:46:49 AM


Acctno: 	0122863683
Channel: 	FlutterWave App
Cr_Amt: 	100000
Dr_Amt: 	0
Remark: 	000015161115094006000091598495  bro LN FASHOLA OMOBOLAJI OLUWASEYI TAOFIK ADEMOLA ADEDEJI  to  FASHOLA OMOBOLAJI OLUWASEYI
Tra_Bal: 	309017.15
Tra_Date: 	11/15/2016 9:39:03 AM
Trans_Type

Tra_Date: 	8/23/2016 1:08:21 AM
Trans_Type: 	Spend to Save  Contribution
Val_Date: 	8/23/2016 1:08:21 AM


Acctno: 	0122863683
Channel: 	NIBSS Instant Payment Outward
Cr_Amt: 	0
Dr_Amt: 	5000
Remark: 	000013160823162057000015656020  TO ACCESS/UTHMAN MOSES KAYODE  from  FASHOLA, OMOBOLAJI OLUWASEYI
Tra_Bal: 	1626.91
Tra_Date: 	8/23/2016 4:20:57 PM
Trans_Type: 	NIBSS Instant Payment Outward
Val_Date: 	8/23/2016 4:20:57 PM


Acctno: 	0122863683
Channel: 	Internet Banking
Cr_Amt: 	5000
Dr_Amt: 	0
Remark: 	trf via Internet Banking
Tra_Bal: 	6626.91
Tra_Date: 	8/23/2016 4:19:59 PM
Trans_Type: 	OWN ACCOUNT TRANSFER
Val_Date: 	8/23/2016 4:19:59 PM


Acctno: 	0122863683
Channel: 	Mobile Banking
Cr_Amt: 	1200
Dr_Amt: 	0
Remark: 	MBANKING -  ANY Account Transfer   from ASUELIME, OSOBASE OSEREME  to  FASHOLA, OMOBOLAJI OLUWASEYI
Tra_Bal: 	1626.91
Tra_Date: 	8/23/2016 2:11:58 PM
Trans_Type: 	TRANSFER BETWEEN CUSTOMERS
Val_Date: 	8/23/2016 2:11:58 PM


Acctno: 	0122863683
Channel: 	USSD
Cr_Amt: 	0
D

Tra_Bal: 	2299.98
Tra_Date: 	5/26/2016 1:32:32 AM
Trans_Type: 	Spend to Save  Contribution
Val_Date: 	5/26/2016 1:32:32 AM


Acctno: 	0122863683
Channel: 	CASH WITHDRAWAL FROM OUR ATM
Cr_Amt: 	0
Dr_Amt: 	2000
Remark: 	-009014- -GTBank A/ Ademola       V/I         LANG
Tra_Bal: 	2359.98
Tra_Date: 	5/26/2016 8:44:57 PM
Trans_Type: 	CASH WITHDRAWAL FROM OUR ATM
Val_Date: 	5/26/2016 8:44:57 PM


Acctno: 	0122863683
Channel: 	Internet Banking
Cr_Amt: 	3000
Dr_Amt: 	0
Remark: 	trf via Internet Banking
Tra_Bal: 	4359.98
Tra_Date: 	5/26/2016 6:46:03 PM
Trans_Type: 	OWN ACCOUNT TRANSFER
Val_Date: 	5/26/2016 6:46:03 PM


Acctno: 	0122863683
Channel: 	Spend to Save  Contribution
Cr_Amt: 	0
Dr_Amt: 	210
Remark: 	Savings from expenses from 205/152573/1/1
Tra_Bal: 	1359.98
Tra_Date: 	5/23/2016 2:59:11 AM
Trans_Type: 	Spend to Save  Contribution
Val_Date: 	5/23/2016 2:59:11 AM


Acctno: 	0122863683
Channel: 	CASH WITHDRAWAL FROM OUR ATM
Cr_Amt: 	0
Dr_Amt: 	7000
Remark: 	-000239- -GTBank St Gregory   

Tra_Bal: 	120742.09
Tra_Date: 	1/11/2016 5:33:48 PM
Trans_Type: 	NIBSS Instant Payment Outward
Val_Date: 	1/11/2016 5:33:48 PM


Acctno: 	0122863683
Channel: 	Mobile Banking
Cr_Amt: 	0
Dr_Amt: 	1000
Remark: 	airtime recharge via MBANKING-101C0000000000108969268-2348081467539
Tra_Bal: 	132742.09
Tra_Date: 	1/8/2016 10:19:05 PM
Trans_Type: 	Airtime Purchase
Val_Date: 	1/8/2016 10:19:05 PM


Acctno: 	0122863683
Channel: 	POS
Cr_Amt: 	0
Dr_Amt: 	2060
Remark: 	-413202- -GRAND SQ. SUPERMARKET  LA           LANG
Tra_Bal: 	133742.09
Tra_Date: 	1/8/2016 9:07:22 PM
Trans_Type: 	POS/WEB PURCHASE TRANSACTION
Val_Date: 	1/8/2016 9:07:22 PM


Acctno: 	0122863683
Channel: 	Internet Banking
Cr_Amt: 	0
Dr_Amt: 	5000
Remark: 	via Internet Banking For you  from FASHOLA, OMOBOLAJI OLUWASEYI  to  ADETUNJI, ABISOLA ODUNAYO
Tra_Bal: 	135802.09
Tra_Date: 	1/6/2016 6:18:00 PM
Trans_Type: 	TRANSFER BETWEEN CUSTOMERS
Val_Date: 	1/6/2016 6:18:00 PM


Acctno: 	0122863683
Channel: 	Mobile Banking
Cr_Amt: 	0
Dr_Amt:

Remark: 	-074689- -GOOGLE *BlackBerry     GOOGLE.COM/CHCAUS  1.36 USD
Tra_Bal: 	6702.13
Tra_Date: 	9/17/2015 10:25:28 AM
Trans_Type: 	POS/WEB PURCHASE TRANSACTION
Val_Date: 	9/17/2015 10:25:28 AM


Acctno: 	0122863683
Channel: 	USSD
Cr_Amt: 	0
Dr_Amt: 	1000
Remark: 	Airtime Recharge-101C0000000000078598798-2348081467539
Tra_Bal: 	7002.13
Tra_Date: 	9/16/2015 5:09:55 PM
Trans_Type: 	Airtime Purchase
Val_Date: 	9/16/2015 5:09:55 PM


Acctno: 	0122863683
Channel: 	NIBSS Instant Payment Outward
Cr_Amt: 	0
Dr_Amt: 	5000
Remark: 	058011150916170907992907684124 for you dear TO AYODELE OPEYEMI YEWANDE  from  FASHOLA, OMOBOLAJI OLUWASEYI
Tra_Bal: 	8002.13
Tra_Date: 	9/16/2015 5:09:08 PM
Trans_Type: 	NIBSS Instant Payment Outward
Val_Date: 	9/16/2015 5:09:08 PM


Acctno: 	0122863683
Channel: 	Internet Banking
Cr_Amt: 	11701.53
Dr_Amt: 	0
Remark: 	trf via Internet Banking Balancing the withdrawals
Tra_Bal: 	13002.13
Tra_Date: 	9/16/2015 4:45:53 PM
Trans_Type: 	OWN ACCOUNT TRANSFER
Val_Date: 	9/16

Convert data to pandas dataframe

In [4]:
data = pd.DataFrame(arrayy)
data

,Acctno,Channel,Cr_Amt,Dr_Amt,Remark,Tra_Bal,Tra_Date,Trans_Type,Val_Date
0,0122863683,Spend to Save Contribution,0,36,Savings from expenses from 205/152573/1/1,225006.35,5/31/2017 4:03:01 AM,Spend to Save Contribution,5/30/2017 4:03:01 AM
1,0122863683,MEDICAL BILL/REFUND,50466.97,0,Medrefund May 2017,225042.35,5/30/2017 3:37:21 PM,MEDICAL BILL/REFUND,5/31/2017 3:37:21 PM
2,0122863683,MISC.,106161.85,0,May 2017 Allow,174575.38,5/30/2017 3:36:03 PM,MISC.,5/31/2017 3:36:03 PM
3,0122863683,MTHLY SALARY,68353.56,0,May 2017 Sal,68413.53,5/30/2017 3:34:13 PM,MTHLY SALARY,5/31/2017 3:34:13 PM
4,0122863683,USSD,0,200,USSD-101CT0000000000361857077-2348081467539,59.97,5/29/2017 1:53:08 PM,Airtime Purchase,5/30/2017 1:53:08 PM
5,0122863683,USSD,0,500,USSD-101CT0000000000360957605-2348081467539,259.97,5/28/2017 12:11:59 AM,Airtime Purchase,5/30/2017 12:11:59 AM
6,0122863683,USSD,0,500,USSD-101CT0000000000360381199-2348081467539,759.97,5/27/2017 10:31:08 AM,Airtime Purchase,5/30/2017 10:31:08 AM
7,0122863683,Spend to Save Contribution,0,6,Savings from expenses from 205/152573/1/1,1259.97,5/26/2017 12:55:47 AM,Spend to Save Contribution,5/25/2017 12:55:47 AM
8,0122863683,USSD,0,200,USSD-101CT0000000000359236868-2349077611129,1265.97,5/25/2017 8:20:42 PM,Airtime Purchase,5/25/2017 8:20:42 PM
9,0122863683,Spend to Save Contribution,0,45,Savings from expenses from 205/152573/1/1,1465.97,5/25/2017 12:13:11 AM,Spend to Save Contribution,5/24/2017 12:13:11 AM


First thing to note is that this is near perfect data because there is a remark for every transaction, this sort of makes our task easy. In the real world, most people dont annotate a remark on every transaction (or at least i speak for myself). 
